In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
def nfl_short(longn):
    #
    import re
    #
    shortn = 'bogus'
    # I: re.I = Ignore case.
    if re.search('(Jacksonville|Jaguars|jax)', longn, re.I): shortn = 'Jaguars'
    if re.search('Carolina Panthers', longn, re.I): shortn = 'Panthers'
    if re.search('Cleveland Browns', longn, re.I): shortn = 'Browns'
    if re.search('Detroit Lions', longn, re.I): shortn = 'lions'
    if re.search('Houston Texans', longn, re.I): shortn = 'Texans'
    if re.search('Indianapolis Colts', longn, re.I): shortn = 'Colts'
    if re.search('Kansas City Chiefs', longn, re.I): shortn = 'Chiefs'
    if re.search('New York Giants', longn, re.I): shortn = 'Giants'
    if re.search('Tennessee Titans', longn, re.I): shortn = 'Titans'
    if re.search('Washington', longn, re.I): shortn = 'Wash'
    if re.search('Arizona Cardinals', longn, re.I): shortn = 'Cardinals'
    if re.search('Seattle Seahawks', longn, re.I): shortn = 'Seahawks'
    if re.search('Baltimore Ravens', longn, re.I): shortn = 'Ravens'
    if re.search('Pittsburgh Steelers', longn, re.I): shortn = 'Steelers'
    if re.search('Tampa Bay Buccaneers', longn, re.I): shortn = 'Bucs'
    if re.search('Las Vegas Raiders', longn, re.I): shortn = 'Raiders'
    if re.search('Cincinnati Bengals', longn, re.I): shortn = 'Bengals'
    if re.search('Dallas Cowboys', longn, re.I): shortn = 'Cowboys'
    if re.search('Minnesota Vikings', longn, re.I): shortn = 'Vikings'
    if re.search('Chicago Bears', longn, re.I): shortn = 'Bears'
    if re.search('Buffalo Bills', longn, re.I): shortn = 'Bills'
    if re.search('Miami Dolphins', longn, re.I): shortn = 'Dolphins'
    if re.search('Philadelphia Eagles', longn, re.I): shortn = 'Eagles'
    if re.search('New Orleans Saints', longn, re.I): shortn = 'Saints'
    if re.search('New York Jets', longn, re.I): shortn = 'Jets'
    if re.search('Atlanta Falcons', longn, re.I): shortn = 'Falcons'
    if re.search('Los Angeles Rams', longn, re.I): shortn = 'Rams'
    if re.search('San Francisco 49ers', longn, re.I): shortn = '49ers'
    if re.search('Denver Broncos', longn, re.I): shortn = 'Broncos'
    if re.search('Green Bay Packers', longn, re.I): shortn = 'Packers'
    if re.search('New England Patriots', longn, re.I): shortn = 'Patriots'
    if re.search('Los Angeles Chargers', longn, re.I): shortn = 'Chargers'
    #
    return(shortn)
#./. def 
#
        
def mrsplit_text(arrv):
    #
    #Ex: ['', '\xa0Thu Sep 9', '8:20 pm', '\xa0Dallas Cowboys', '\xa0Tampa Bay Buccaneers', '']
    import re
    #
    i=0        
    key  = "bogus"
    home = "bogus"
    away = "bogus"
    sweek = 0
    #
    for e in arrv:
        e = e.replace(u'\xa0', '')
        if i == 1:
            if re.match("(Thu|Sun|Mon)", e):                
                key = e
                if re.match("(Thu|Sun)", e):
                    sweek = 1
            else:
                key = "Zzz"
                #arr.append("Zzz")
                
        if i > 1:            
            if re.match("\w+\s+", e):
                if i == 3: away = nfl_short(e)
                if i == 4: home = nfl_short(e)
                #arr.append(e)
        #./. if i
        #
        i += 1
    #./ eo for
    #
    mydict = {"Key":key, "Week":0, "Away":away, "Home":home}
    return(mydict, sweek)
#./. def 
#

In [3]:
#url = "http://www.epicurious.com/search/Tofu Chili" # MR: Do not need %20, we use space
url = "https://fftoday.com/nfl/schedule.php"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [4]:
results_page = BeautifulSoup(response.content,'lxml')
#print(results_page.prettify())

In [6]:
#
# I: Find 1st center, 2nd table, then all <tr> after abnalysis of html in site (firefox->web delveloper)
center_tag = results_page.find('center') 
all_tbl_tags = center_tag.find_all('table')
i=0
for t in all_tbl_tags:
    if (i == 1):
        mtable = t
        #print("TXT:", t)        
    #./.
    #
    i += 1
#./. for
#
all_tr = mtable.find_all('tr', class_="bodycontent")
#
# I: Loop thru <tr> and extract the games.
season = "2023"
i      = 0
iday   = "bog"
week   = 0
arr_games = []
#
for t in all_tr:
    #D: print("TXT:", t.get_text()) 
    tvar = t.get_text()
    tvar = tvar.replace("\n", ";")
    arrv = tvar.split(";")
    (tdict, sweek) = mrsplit_text(arrv)
    #print ("TXT:", sweek, " ", arrv, tdict)    
    #
    # Choosing start of the week
    if sweek == 1:    
        if re.match("Thu", tdict['Key']):
            iday = "Thu"
            week += 1            
        elif re.match("Sun", tdict['Key']) and iday == "Thu":
            iday  = "bog"
            sweek = 0
        elif re.match("Sun", tdict['Key']) and iday != "Thu":
            iday = "Sun"
            week += 1
    #./.
    tdict['Week'] = week
    tdict['Iva'] = np.nan
    tdict['Gigo'] = np.nan
    tdict['Andy'] = np.nan    
    tdict['Flaco'] = np.nan
    tdict['Dad'] = np.nan
    #D: if (week > 14): print ("TXT:", sweek, iday, " ", tdict)    
    if (sweek == 1): 
        tdicth = {"Key":"Header", "Week":week, "Away":"Week "+str(week)+" "+tdict['Key'], "Home":"Home", 
              'Iva':"Iva", 'Gigo':'Gigo', 'Andy':'Andy', 'Flaco':'Flaco', 'Dad':'Dad'}
        # D: print ("TXTH:", sweek, iday, " ", tdicth, arrv)
        # I: if tdicth is outside of the loop, tdicth remains a single memory entry. This means that 
        #    the array is adding a link to a static dict where the last entry gets propagated to all the eles.
        arr_games.append(tdicth) 
        #D: print("\t", arr_games,"\n")
    #./. if
    #
    arr_games.append(tdict)
    i += 1
#./. for
#
#
# I: Create excel file. Change font to 16, center per row, 
#    adjust cell width, skip printing 1st 2 cols => send2 kitchen
games_df = pd.DataFrame(arr_games)
outf = "NFL_season_"+season+".xlsx"
games_df.to_excel(outf, sheet_name='Sheet', index=False) 
games_df
#
#

,Key,Week,Away,Home,Iva,Gigo,Andy,Flaco,Dad
0,Header,1,Week 1 Thu Sep 7,Home,Iva,Gigo,Andy,Flaco,Dad
1,Thu Sep 7,1,lions,Chiefs,NaN,NaN,NaN,NaN,NaN
2,Sun Sep 10,1,Cardinals,Wash,NaN,NaN,NaN,NaN,NaN
3,Zzz,1,Panthers,Falcons,NaN,NaN,NaN,NaN,NaN
4,Zzz,1,Bengals,Browns,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
285,Zzz,18,Jets,Patriots,NaN,NaN,NaN,NaN,NaN
286,Zzz,18,Eagles,Giants,NaN,NaN,NaN,NaN,NaN
287,Zzz,18,Steelers,Ravens,NaN,NaN,NaN,NaN,NaN
288,Zzz,18,Seahawks,Cardinals,NaN,NaN,NaN,NaN,NaN
